© 2024 Brian Butka. All rights reserved.

In [59]:
def binary_addition(a, b):
    max_len = max(len(a), len(b))
    a = a.zfill(max_len)
    b = b.zfill(max_len)
    result = ''
    carry = 0

    for i in range(max_len - 1, -1, -1):
        total = carry
        total += 1 if a[i] == '1' else 0
        total += 1 if b[i] == '1' else 0
        result = ('1' if total % 2 == 1 else '0') + result
        carry = 0 if total < 2 else 1

    if carry != 0:
        result = '1' + result

    return result.zfill(max_len), carry

def binary_subtraction(a, b):
    max_len = max(len(a), len(b))
    a = a.zfill(max_len)
    b = b.zfill(max_len)
    complement_b = ''.join('1' if x == '0' else '0' for x in b)
    sum_a_complement_b, _ = binary_addition(a, complement_b)
    result, carry = binary_addition(sum_a_complement_b, '1')
    return result[-max_len:], carry

def non_restoring_division(dividend, divisor):
    n = len(dividend)
    A = '0' * n  # Initialize A with n bits (5 bits for A)
    Q = dividend
    M = divisor.zfill(n)
    M_complement, _ = binary_subtraction('0' * n, M)

    steps = []

    def record_step(action, iteration=None, highlight_sign=False, highlight_carry=False, carry=None, full_A=None):
        carry_display = f"\033[92m{carry}\033[0m" if carry is not None else ''
        if full_A:
            A_highlighted = full_A[:n]
        else:
            sign_bit = f"\033[91m{A[0]}\033[0m" if highlight_sign else A[0]
            A_highlighted = sign_bit + A[1:]

        Q_highlighted = Q[:-1] + (f"\033[92m{Q[-1]}\033[0m" if highlight_carry else Q[-1])
        iteration_display = f"N={iteration}" if iteration is not None else ""
        steps.append(f"{carry_display}\t{A_highlighted[-4:]} {Q_highlighted} {action} {iteration_display}")  # Show the rightmost 4 bits of A

    sign_bit_initial = f"\033[91m{A[0]}\033[0m"
    initial_A = sign_bit_initial + A[1:]
    steps.append(f"\t{initial_A} {Q} Sign bit of A=0")

    for i in range(n):
        # Shift left A and Q
        A = A[1:] + Q[0]
        Q = Q[1:] + '0'
        record_step("Shift Left A_Q")

        # Check the sign bit of A and perform addition or subtraction
        if A[0] == '1':  # A is negative
            full_A, carry = binary_addition(A, M)
            A = full_A[-n:]
            record_step("A = A + M", carry=carry, full_A=A)
        else:  # A is positive
            full_A, carry = binary_addition(A, M_complement)
            A = full_A[-n:]
            record_step("A = A - M", carry=carry, full_A=A)

        #Set Q[0] to the carry out of the addition or subtraction
        Q = Q[:-1] + str(carry)
        record_step(f"Q[0] = Carry Out", highlight_carry=True, iteration=n-i)


        # Insert a horizontal divider for separation
        steps.append("-" * 40)

        # Copy previous A and Q for the new Sign bit of A line
        A_copy = A
        Q_copy = Q

        # Highlight the sign bit for the next iteration
        sign_bit_next = f"\033[91m{A_copy[0]}\033[0m"
        steps.append(f"\t{sign_bit_next}{A_copy[1:]} {Q_copy} Sign bit of A")

    # Final correction if A is negative
    if A[0] == '1':
        full_A, _ = binary_addition(A, M)
        A = full_A[-n:]
        record_step("Final correction, A = A + M", full_A=A)
    record_step("Done")

    print("Step\tCarry_out\tA\tQ\tComment")
    for idx, step in enumerate(steps):
        if step.strip():  # Skip printing blank lines
            print(f"{idx}\t{step}")

    print(f"\nResult: {Q}  Remainder: {A}")
    print(f"Result (Decimal): {int(Q, 2)}  Remainder (Decimal): {int(A, 2)}")


# Prompt user for input
dividend = input("Enter the dividend (in binary, e.g., 01111): ").strip()
divisor = input("Enter the divisor (in binary, e.g., 00010): ").strip()

non_restoring_division(dividend, divisor)


Enter the dividend (in binary, e.g., 01111): 1110
Enter the divisor (in binary, e.g., 00010): 0011
Step	Carry_out	A	Q	Comment
0		0000 1110 Sign bit of A=0
1		0001 1100 Shift Left A_Q 
2	0	1110 1100 A = A - M 
3		1110 1100 Q[0] = Carry Out N=4
4	----------------------------------------
5		1110 1100 Sign bit of A
6		1101 1000 Shift Left A_Q 
7	1	0000 1000 A = A + M 
8		0000 1001 Q[0] = Carry Out N=3
9	----------------------------------------
10		0000 1001 Sign bit of A
11		0001 0010 Shift Left A_Q 
12	0	1110 0010 A = A - M 
13		1110 0010 Q[0] = Carry Out N=2
14	----------------------------------------
15		1110 0010 Sign bit of A
16		1100 0100 Shift Left A_Q 
17	0	1111 0100 A = A + M 
18		1111 0100 Q[0] = Carry Out N=1
19	----------------------------------------
20		1111 0100 Sign bit of A
21		0010 0100 Final correction, A = A + M 
22		0010 0100 Done 

Result: 0100  Remainder: 0010
Result (Decimal): 4  Remainder (Decimal): 2


In [ ]:

    %%javascript
    function hideCode(){
        var code_cells = document.querySelectorAll('div.input');
        for (var i = 0; i < code_cells.length; i++) {
            code_cells[i].style.display = 'none';
        }
    }
    hideCode();
    